In [2042]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import numpy as np
from datetime import date, timedelta
pd.options.mode.chained_assignment = None

In [2043]:
%cd /Users/sjneelyf/Desktop/Analisis_de_datos
df = pd.read_csv('Cuarentenas-Totales.csv')

/Users/sjneelyf/Desktop/Analisis_de_datos


In [2044]:
df

,ID,Nombre,Estado,Alcance,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Detalle,Superficie en m2,Perímetro en m,Region,n_REGION
0,1,Arica,Histórica,Área Urbana Completa,2020-04-16 22:00:00,2020-05-15 22:00:00,15101,Corresponde al Radio Urbano de la Ciudad de Ar...,28050418.80,48722.30001,15,R15
1,2,Santiago Norte,Histórica,Sector Específico,2020-04-13 05:00:00,2020-05-05 22:00:00,13101,Corresponde a la extensión de la cuarentena en...,19939218.57,21599.35134,13,R13
2,3,Puente Alto Poniente,Histórica,Sector Específico,2020-04-09 22:00:00,2020-05-08 22:00:00,13201,Corresponde al sector delimitado por la Av. Co...,42364268.21,30428.05152,13,R13
3,4,El Bosque,Histórica,Comuna completa,2020-04-16 22:00:00,2020-09-21 05:00:00,13105,Aplicada a la totalidad de la comuna. Se incor...,20622713.23,18788.87075,13,R13
4,5,San Bernardo Nororiente,Histórica,Sector Específico,2020-04-16 22:00:00,2020-05-08 22:00:00,13401,Corresponde al sector delimitado por los limit...,16326216.83,24929.14572,13,R13
...,...,...,...,...,...,...,...,...,...,...,...,...
182,183,Angol,Activa,NaN,2020-12-10 05:00:00,2020-12-25 22:00:00,9201,NaN,NaN,NaN,9,R09
183,184,Cabo de Hornos,Activa,NaN,2020-12-10 05:00:00,2020-12-25 22:00:00,12201,NaN,NaN,NaN,12,R12
184,185,Los Ángeles,Activa,NaN,2020-12-12 05:00:00,2020-12-25 22:00:00,8301,NaN,NaN,NaN,8,R08
185,186,Paillaco,Histórica,NaN,2020-12-12 05:00:00,2020-12-23 05:00:00,14107,NaN,NaN,NaN,14,R14


In [2045]:
# Elimino columnas que no sirven:
df = df.drop(['ID', 
         'Detalle',
         'Superficie en m2', 
         'Perímetro en m',
         'n_REGION', 'Alcance',
         'Estado'], 
          axis =1)

In [2046]:
# Renombrar codigo CUT comuna
df.rename(columns = {'Código CUT Comuna': 'codigo_comuna'}, inplace = True)

In [2047]:
# Cargar archivo poblacion.xls para luego cruzar con df y obtener poblacion por comuna:
pop = pd.read_excel("poblacion.xls", sheet_name="Comuna")

# Selecciono columnas de interes:
pop = pop[['TOTAL POBLACIÓN EFECTIVAMENTE CENSADA', 'EDAD', 'CÓDIGO COMUNA']]

# Cambio nombre a columnas para mejor manejo:
pop.rename(columns = {'TOTAL POBLACIÓN EFECTIVAMENTE CENSADA': 'poblacion', 'CÓDIGO COMUNA': 'codigo_comuna'}, inplace = True)

# El archivo muestra poblacion por edades y muestra el total una fila mas abajo, 
# se debe seleccionar solo esas filas para cada comuna:
pop = pop.loc[pop['EDAD'] == 'Total Comuna']

# Elimino columna EDAD que ya no me sirve:
pop = pop.drop(['EDAD'], axis = 1)


In [2048]:
pop['codigo_comuna'] = pd.to_numeric(pop['codigo_comuna'])
#pop.info()

In [2049]:
# Cruzo df con pop por codigo_comuna:
dfpop = df.merge(pop, on = 'codigo_comuna', how ='left')
#dfpop.info()

In [2050]:
# Elimino hras de inicio y termino
dfpop['Fecha de Inicio'] = dfpop['Fecha de Inicio'].apply(lambda x: x.split(' ')[0])
dfpop['Fecha de Término'] = dfpop['Fecha de Término'].apply(lambda x: x.split(' ')[0])

# Cambio de formato las fechas:
dfpop['Fecha de Inicio'] = pd.to_datetime(dfpop['Fecha de Inicio'], format='%Y/%m/%d')
dfpop['Fecha de Término'] = pd.to_datetime(dfpop['Fecha de Término'], format='%Y/%m/%d')

# Renombro columnas para mejor manejo:
dfpop.rename(columns = {'Fecha de Inicio': 'inicio', 'Fecha de Término': 'termino'}, inplace = True)


dfpop


,Nombre,inicio,termino,codigo_comuna,Region,poblacion
0,Arica,2020-04-16,2020-05-15,15101,15,221364
1,Santiago Norte,2020-04-13,2020-05-05,13101,13,404495
2,Puente Alto Poniente,2020-04-09,2020-05-08,13201,13,568106
3,El Bosque,2020-04-16,2020-09-21,13105,13,162505
4,San Bernardo Nororiente,2020-04-16,2020-05-08,13401,13,301313
...,...,...,...,...,...,...
182,Angol,2020-12-10,2020-12-25,9201,9,53262
183,Cabo de Hornos,2020-12-10,2020-12-25,12201,12,2063
184,Los Ángeles,2020-12-12,2020-12-25,8301,8,202331
185,Paillaco,2020-12-12,2020-12-23,14107,14,20188


In [2051]:
# Fecha inicio df
idf = min(dfpop['inicio'])
idf

Timestamp('2020-03-12 00:00:00')

In [2052]:
#Fecha fin df
fdf = max(dfpop['termino'])
fdf

Timestamp('2020-12-25 00:00:00')

In [2053]:
fechas = [idf+timedelta(days=x) for x in range((fdf-idf).days)]
fechas = pd.DataFrame([fechas])
# Hacer primera fila columnas:
fechas.columns = fechas.iloc[0]

#Eliminar primera fila:
fechas = fechas.drop([0], axis=0)
fechas

,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,...,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24


In [2054]:
# Concatenar df con fechas:
dfpop = pd.concat([dfpop, fechas], axis = 1)

dfpop

,Nombre,inicio,termino,codigo_comuna,Region,poblacion,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,2020-03-15 00:00:00,...,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00
0,Arica,2020-04-16,2020-05-15,15101,15,221364,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Santiago Norte,2020-04-13,2020-05-05,13101,13,404495,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Puente Alto Poniente,2020-04-09,2020-05-08,13201,13,568106,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,El Bosque,2020-04-16,2020-09-21,13105,13,162505,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,San Bernardo Nororiente,2020-04-16,2020-05-08,13401,13,301313,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Angol,2020-12-10,2020-12-25,9201,9,53262,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
183,Cabo de Hornos,2020-12-10,2020-12-25,12201,12,2063,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
184,Los Ángeles,2020-12-12,2020-12-25,8301,8,202331,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
185,Paillaco,2020-12-12,2020-12-23,14107,14,20188,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [2055]:
# Agregar fila con fechas

dfpop.loc[dfpop.iloc[-1].name + 1,:] = np.nan
#dfpop

In [2056]:
# Move target row to first element of list.
target_row = 187
idx = [target_row] + [i for i in range(len(dfpop)) if i != target_row]
dfpop = dfpop.iloc[idx]
# Reset index
dfpop.reset_index(inplace = True)
dfpop = dfpop.drop(['index'], axis = 1)
dfpop

,Nombre,inicio,termino,codigo_comuna,Region,poblacion,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,2020-03-15 00:00:00,...,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00
0,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Arica,2020-04-16,2020-05-15,15101.0,15.0,221364.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Santiago Norte,2020-04-13,2020-05-05,13101.0,13.0,404495.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Puente Alto Poniente,2020-04-09,2020-05-08,13201.0,13.0,568106.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,El Bosque,2020-04-16,2020-09-21,13105.0,13.0,162505.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Angol,2020-12-10,2020-12-25,9201.0,9.0,53262.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
184,Cabo de Hornos,2020-12-10,2020-12-25,12201.0,12.0,2063.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
185,Los Ángeles,2020-12-12,2020-12-25,8301.0,8.0,202331.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
186,Paillaco,2020-12-12,2020-12-23,14107.0,14.0,20188.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [2057]:
# Primera fila con valores de las columnas:
for i in range(0,294):
    dfpop.iloc[0, i] = df1.columns[i]


dfpop

,Nombre,inicio,termino,codigo_comuna,Region,poblacion,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,2020-03-15 00:00:00,...,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00
0,index,Nombre,inicio,termino,codigo_comuna,Region,poblacion,2020-03-12,2020-03-13,2020-03-14,...,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23
1,Arica,2020-04-16 00:00:00,2020-05-15 00:00:00,15101.0,15.0,221364.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Santiago Norte,2020-04-13 00:00:00,2020-05-05 00:00:00,13101.0,13.0,404495.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Puente Alto Poniente,2020-04-09 00:00:00,2020-05-08 00:00:00,13201.0,13.0,568106.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,El Bosque,2020-04-16 00:00:00,2020-09-21 00:00:00,13105.0,13.0,162505.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Angol,2020-12-10 00:00:00,2020-12-25 00:00:00,9201.0,9.0,53262.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
184,Cabo de Hornos,2020-12-10 00:00:00,2020-12-25 00:00:00,12201.0,12.0,2063.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
185,Los Ángeles,2020-12-12 00:00:00,2020-12-25 00:00:00,8301.0,8.0,202331.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
186,Paillaco,2020-12-12 00:00:00,2020-12-23 00:00:00,14107.0,14.0,20188.0,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [2058]:
# Un df por region para mejor manejo:
df1 = dfpop.loc[dfpop['Region'] == 1] 
df2 = dfpop.loc[dfpop['Region'] == 2]
df3 = dfpop.loc[dfpop['Region'] == 3]
df4 = dfpop.loc[dfpop['Region'] == 4]
df5 = dfpop.loc[dfpop['Region'] == 5]
df6 = dfpop.loc[dfpop['Region'] == 6]
df7 = dfpop.loc[dfpop['Region'] == 7]
df8 = dfpop.loc[dfpop['Region'] == 8]
df9 = dfpop.loc[dfpop['Region'] == 9]
df10 = dfpop.loc[dfpop['Region'] == 10]
df11 = dfpop.loc[dfpop['Region'] == 11]
df12 = dfpop.loc[dfpop['Region'] == 12]
df13 = dfpop.loc[dfpop['Region'] == 13]
df14 = dfpop.loc[dfpop['Region'] == 14]
df15 = dfpop.loc[dfpop['Region'] == 15]

In [2059]:
df1.reset_index(inplace = True)

In [2060]:
# Asignar poblacion a cada fecha que haya estado en cuarentena, para cada comuna:

for x in range(0,len(df1)):
    for i in range(7,295):
        if df1.loc[x]['inicio'] <= df1.columns[i] <= df1.loc[x]['termino']:
            df1.iloc[x,i] = df1.loc[x]['poblacion']
        else:
            pass
df1   

,index,Nombre,inicio,termino,codigo_comuna,Region,poblacion,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,...,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00
0,57,Alto Hospicio,2020-05-15 00:00:00,2020-10-03 00:00:00,1107.0,1.0,108375.0,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,58,Iquique,2020-05-15 00:00:00,2020-10-03 00:00:00,1101.0,1.0,191468.0,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,80,Pozo Almonte (Radio Urbano),2020-06-12 00:00:00,2020-09-28 00:00:00,1401.0,1.0,15711.0,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [2062]:
# Luego falta crear una fila al final llamada total, esta indicará la suma de las poblaciones en cuarentena
# de cada comuna para cada día.
df1.loc['total'] = df1.sum()
df1
# Ubicar dia con mas personas en cuarentena: 

# Fin

,index,Nombre,inicio,termino,codigo_comuna,Region,poblacion,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,...,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,2020-12-22 00:00:00,2020-12-23 00:00:00,2020-12-24 00:00:00
0,57,Alto Hospicio,2020-05-15 00:00:00,2020-10-03 00:00:00,1107.0,1.0,108375.0,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58,Iquique,2020-05-15 00:00:00,2020-10-03 00:00:00,1101.0,1.0,191468.0,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80,Pozo Almonte (Radio Urbano),2020-06-12 00:00:00,2020-09-28 00:00:00,1401.0,1.0,15711.0,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total,390,Alto HospicioIquiquePozo Almonte (Radio Urbano...,NaN,NaN,7218.0,6.0,631108.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
